In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from config import *
from load_data import fetch_data, fetch_trajectory
from match_rounds import find_match_round_dtw, find_match_round_dtw_kmp
from utils import compute_stat, interplate_and_align

# DataLoader

In [2]:
import os

file_list = os.listdir(DATA_DIR)
file_list = list(filter(lambda x: "AD" in x, file_list))
file_list

['24071512_AD.csv',
 '24070902_AD.csv',
 '24071009_AD.csv',
 '24072825_AD.csv',
 '24070904_AD.csv',
 '24071617_AD.csv',
 '24071721_AD.csv',
 '24071619_AD.csv',
 '24071615_AD.csv',
 '24071011_AD.csv',
 '24070906_AD.csv',
 'Test_AD.csv',
 '24071513_AD.csv',
 '24072926_AD.csv',
 '24071008_AD.csv',
 '24070901_AD.csv',
 '24072023_AD.csv',
 '24071616_AD.csv',
 '24070905_AD.csv',
 '24070907_AD.csv',
 '24071010_AD.csv',
 '24071618_AD.csv']

In [3]:
drop_list = ['pingpang.csv', 'tennis.csv', '.DS_Store', 'ControlGroupInfo.xlsx',]

In [4]:
all_data = fetch_data(DATA_DIR, file_list, drop_list)
ball_data = fetch_trajectory(DATA_DIR)

# Feature Extraction

## Saccade Feature

In [5]:
def compute_saccade_path_lite(df:pd.DataFrame):
    _speeds = [0]
    _angles = [0]
    indices = list(df.index)
    _dura = (1 * EYE_SAMPLE_TIME ) / 1000.0
    df["Screen.x"] = df["Screen.x"] * VR_SCALE
    df["Screen.y"] = df["Screen.y"] * VR_SCALE

    for _i in range(1, len(indices)):
        row_1 = df.loc[indices[_i - 1], :]
        row_2 = df.loc[indices[_i], :]

        _dist = np.sqrt((row_1["Screen.x"] - row_2["Screen.x"])**2 + (row_1["Screen.y"] - row_2["Screen.y"])**2)
        _angle = np.arctan(_dist / VR_ZDIST) / np.pi * 180

        _angles.append(_angle)
        _speeds.append(np.divide(_angle, _dura))

    return _speeds, _angles

## Trajectory Features

In [6]:

def extract_saccade_features_lite(round_index, data:pd.DataFrame, ball_data_df:pd.DataFrame):
    _fea = {}

    _speeds, _angles = compute_saccade_path_lite(data)
    _fea = {**_fea, **compute_stat("SaccadeSpeed", _speeds)}
    _fea = {**_fea, **compute_stat("SaccadeAngel", _angles)}

    round_start_index = ball_data_df[ball_data_df["round"]==round_index].index[0]
    round_dura = ball_data_df[ball_data_df["round"]==round_index].shape[0]
    eye_start_index = data.index[0]
    _fea["SaccadeDelay"] = (eye_start_index-round_start_index) * EYE_SAMPLE_TIME
    _fea["SaccadeDelayPercent"] = ((eye_start_index-round_start_index) / round_dura) * EYE_SAMPLE_TIME

    # _fea["SaccadeTimeShift"] = (np.argmax(_speeds) - (data.shape[0] // 2)) * EYE_SAMPLE_TIME

    # bias_res = compute_drop_point_bias_pred(
    #     eye_data=data,
    #     eye_index=data.index[np.argmax(_speeds)],
    #     ball_data=ball_data_df,
    #     ball_round=round_index
    # )
    # _fea["SaccadeDropBiasPred_Init"] = bias_res[0] * VR_SCALE
    # _fea["SaccadeDropBiasPred_Final"] = bias_res[1] * VR_SCALE

    # bias_res = compute_drop_point_bias_delay(
    #     eye_data=data,
    #     eye_index=data.index[np.argmax(_speeds)],
    #     ball_data=ball_data_df,
    #     ball_round=round_index
    # )
    # _fea["SaccadeDropBiasDelay_Init"] = bias_res[0] * VR_SCALE
    # _fea["SaccadeDropBiasDelay_Final"] = bias_res[1] * VR_SCALE
    return _fea

In [7]:
from utils import compute_dtw

def extract_trajectory_lite(round_index, data:pd.DataFrame, ball_data_df):
    _fea = {}
    
    _ball_df = ball_data_df[ball_data_df["round"]==round_index]
    _dtw = compute_dtw(
            line_1=data.loc[:, ["Screen.x", "Screen.y"]],
            line_2=_ball_df.loc[:, ["Ball.x", "Ball.y"]],
            # scale_to_percentage=True,
            # scale_metrics="start"
        )
    
    _fea["TrajectoryDTW"] = _dtw

    return _fea

## Main extrat features function

In [8]:
def extract_features_round(rounds:dict, data_df:pd.DataFrame, ball_data_df:pd.DataFrame):
    data_df["frame"] = data_df.index
    aligned_df = interplate_and_align(data_df, ball_data_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)

    res = {}
    for round_index, round_indices in rounds.items():
        round_res = extract_saccade_features_lite(
            round_index,
            aligned_df.loc[round_indices, ["Screen.x", "Screen.y"]],
            aligned_df.loc[:, ["Ball.x", "Ball.y", "round"]]
            )
        
        traj_res = extract_trajectory_lite(
            round_index,
            aligned_df.loc[round_indices, ["Screen.x", "Screen.y"]],
            aligned_df.loc[:, ["Ball.x", "Ball.y", "round"]]
            )
        res[round_index] = {**round_res, **traj_res}
    return res

In [9]:
from utils import max_circle_radius

def threshold_find_match_round_dtw(eye_data:pd.DataFrame, ball_data_df:pd.DataFrame, order, scale_raw_data, mode="fast", dtw_th=1, dist_th=10):
    if mode=="fast":
        rounds, dtw_res = find_match_round_dtw_kmp(eye_data_df=eye_data, ball_data_df=ball_data_df, order=order, scale_raw_data=scale_raw_data)
    elif mode=="greedy":
        rounds, dtw_res = find_match_round_dtw(eye_data_df=eye_data, ball_data_df=ball_data_df, order=order, scale_raw_data=scale_raw_data)

    res = {}
    for _round, _round_index in rounds.items():
        if (max_circle_radius(eye_data.loc[_round_index, :]) >= dist_th) and (dtw_res[_round] <= dtw_th):
            res[_round] = _round_index.to_list()

    return res

In [10]:
def extract_features(data, ball_data):
    data_df = pd.DataFrame(data).T
    data_df.ffill(inplace=True)
    data_df.bfill(inplace=True)

    ball_data_df = pd.DataFrame(ball_data)
    
    # match_rounds = label_round_hit(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id)

    match_rounds = threshold_find_match_round_dtw(data_df.copy(), ball_data_df.copy(), order=0, scale_raw_data=True, mode="fast")
    # match_rounds = find_match_round_hit(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id, time_range=7, dist=300)
    saccade_features = extract_features_round(match_rounds, data_df.copy(), ball_data_df.copy())
    
    return match_rounds, saccade_features

    # saccade_features = extract_saccade_features(match_rounds, data_df.loc[:, ["Screen.x", "Screen.y"]], video_id)
    # trajectory_features = extract_trajectory(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id, scale_to_percentage=True)

    # return {**saccade_features, **trajectory_features}

# Extract Features of all Participants

In [11]:
all_people_fea = {}
all_people_rounds = {}
video_list= []

for _person in all_data.keys():
    # if not _person == "24071512_AD": continue
    _person_fea = {}
    _person_rounds = {}

    for _video in all_data[_person].keys():
        if not _video == "p7": continue
        if not _video in video_list: video_list.append(_video)
        
        match_rounds, fea_res = extract_features(data=all_data[_person][_video], ball_data=ball_data[_video.split("_")[0]])
        if fea_res:
            _person_fea[_video] = fea_res
            _person_rounds[_video] = match_rounds

    all_people_fea[_person] = _person_fea
    all_people_rounds[_person] = _person_rounds

In [12]:
def single_video_res(people_fea_res, video_id):
    res = {}
    for _p, person_fea in people_fea_res.items():
        try:
            for _r, _fea in person_fea[video_id].items():
                res[f"{_p}-{_r}"] = _fea
        except:
            continue
    return res


In [13]:
def single_person_res(people_fea_res, people_id):
    res = {}
    for _v, video_fea in people_fea_res[people_id].items():
        try:
            for _r, _fea in video_fea.items():
                res[f"{_v}-{_r}"] = _fea
        except:
            continue
    return res

In [14]:
def single_person_rounds(people_rounds_res, people_id):
    res = {}
    for _v, video_rounds in people_rounds_res[people_id].items():
        try:
            for _r, _fea in video_rounds.items():
                res[f"{_v}-{_r}"] = _fea
        except:
            continue
    return res

In [16]:
person_fea = single_person_res(people_fea_res=all_people_fea, people_id="24071512_AD")
# pd.DataFrame(person_fea).T.to_csv("24071512_AD_dtw_fast_rounds.csv")

In [17]:
video_id = video_list[0]
print(video_id)
video_res = single_video_res(people_fea_res=all_people_fea, video_id=video_id)
pd.DataFrame(video_res).T

p7


,SaccadeSpeed_Mean,SaccadeSpeed_Max,SaccadeSpeed_Min,SaccadeSpeed_Std,SaccadeAngel_Mean,SaccadeAngel_Max,SaccadeAngel_Min,SaccadeAngel_Std,SaccadeDelay,SaccadeDelayPercent,TrajectoryDTW
24071512_AD-3.0,82.640553,413.202767,0.0,165.281107,1.652811,8.264055,0.0,3.305622,600.0,20.000000,422.573917
24071512_AD-8.0,304.883800,654.553153,0.0,269.090155,6.097676,13.091063,0.0,5.381803,0.0,0.000000,383.265395
24071512_AD-10.0,162.601117,221.936778,0.0,93.971469,3.252022,4.438736,0.0,1.879429,1020.0,34.000000,436.774472
24071512_AD-18.0,178.374241,526.025556,0.0,208.771884,3.567485,10.520511,0.0,4.175438,860.0,34.400000,387.935344
24071512_AD-21.0,198.640895,385.736076,0.0,162.337505,3.972818,7.714722,0.0,3.246750,1040.0,29.714286,263.899206
24071009_AD-2.0,145.120984,287.096155,0.0,105.141863,2.902420,5.741923,0.0,2.102837,1020.0,26.153846,449.428613
24071009_AD-3.0,67.554320,357.973618,0.0,131.025338,1.351086,7.159472,0.0,2.620507,140.0,4.666667,266.550089
24071009_AD-4.0,46.390847,84.463117,0.0,34.980028,0.927817,1.689262,0.0,0.699601,420.0,21.000000,410.719583
24071009_AD-15.0,58.775913,205.685419,0.0,68.921618,1.175518,4.113708,0.0,1.378432,1100.0,25.000000,668.056855
24070904_AD-22.0,83.860189,167.633435,0.0,71.947683,1.677204,3.352669,0.0,1.438954,680.0,22.666667,425.209707


# Save Results to Output

In [18]:
if not os.path.exists("output"):
    os.mkdir("output")

In [20]:
import json

for file in file_list:
    if "Test" in file: continue
    people_id = file.split(".")[0]
    person_fea = single_person_res(people_fea_res=all_people_fea, people_id=people_id)
    person_rounds = single_person_rounds(people_rounds_res=all_people_rounds, people_id=people_id)

    out_dir = f"output/{people_id}"
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    pd.DataFrame(person_fea).T.to_csv(f"{out_dir}/dtw_fast_features.csv")
    with open(f"{out_dir}/dtw_fast_rounds.json", "w") as f:
        json.dump(person_rounds, f)


In [21]:
person_fea

{'p7-1.0': {'SaccadeSpeed_Mean': 100.19448569050886,
  'SaccadeSpeed_Max': 455.44894339932097,
  'SaccadeSpeed_Min': 0.0,
  'SaccadeSpeed_Std': 156.36009214947146,
  'SaccadeAngel_Mean': 2.003889713810177,
  'SaccadeAngel_Max': 9.10897886798642,
  'SaccadeAngel_Min': 0.0,
  'SaccadeAngel_Std': 3.1272018429894297,
  'SaccadeDelay': 0.0,
  'SaccadeDelayPercent': 0.0,
  'TrajectoryDTW': 684.3761613960843},
 'p7-4.0': {'SaccadeSpeed_Mean': 31.953487114776692,
  'SaccadeSpeed_Max': 82.6972536991272,
  'SaccadeSpeed_Min': 0.0,
  'SaccadeSpeed_Std': 30.74822231665599,
  'SaccadeAngel_Mean': 0.6390697422955338,
  'SaccadeAngel_Max': 1.653945073982544,
  'SaccadeAngel_Min': 0.0,
  'SaccadeAngel_Std': 0.6149644463331199,
  'SaccadeDelay': 340.0,
  'SaccadeDelayPercent': 17.0,
  'TrajectoryDTW': 503.5479326134089}}